# 데이터 불러오기

In [1]:
!pip install sqlalchemy

In [2]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd

In [6]:
endpoint = 'dao.c51deksujiip.ap-northeast-2.rds.amazonaws.com'
schema_name = 'dao'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, schema_name )
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

In [7]:
# 교통량 데이터 불러오기
sql = "SELECT * FROM {}".format("교통량_2021")
df = pd.read_sql(sql, db_connection)

In [8]:
# 메인 데이터 불러오기
sql2 = "select * from {}".format("MainData")
df2 = pd.read_sql(sql2, db_connection)

# 일정 범위 이내에 교통량 비교하기

In [7]:
!pip install haversine

## 필요한 패키지 불러오기

In [10]:
from haversine import haversine
import numpy as np

In [12]:
df.columns

Index(['연도', '도로종류', '지점번호', '호선', '구간', '차로', '도/시', '시/군', '읍/면', '동/리',
       '구분', 'AADT', 'XCODE', 'YCODE'],
      dtype='object')

## 7.8km 이내에 교통량 찾기 

In [13]:
count = 0
index = []
for i in range(len(df2)):
    for j in range(len(df)):
        dist = haversine((df2[:]["위도"][i],df2[:]["경도"][i]), (df[:]["XCODE"][j],df[:]["YCODE"][j]), unit = 'km')
        count +=1
        if dist <= 7.82 : #각 구간간 거리평균이 7.82
            index.append([i,j]) # 7.8km 이내에 있는 교통량의 인덱스 리스트에 넣기            

In [14]:
# 리스트 데이터 프레임으로 만들기
index2 = pd.DataFrame(index) # 0 : 생태통로 index, 1: 교통량 index
index2

,0,1
0,0,1180
1,0,1182
2,0,1183
3,0,1649
4,0,1651
...,...,...
4338,496,1743
4339,496,2535
4340,496,2538
4341,496,2590


In [15]:
# 교통량 값만 불러오기
a=pd.DataFrame(df["AADT"].iloc[index2[1]])
a = a.reset_index()

In [16]:
# 교통량 값이랑 인덱스 데이터 프레임이랑 합치기 
index3 = pd.concat([index2, a],axis =1)

# 하나의 생태통로에 여러개의 교통량이 관측되면 평균값으로 계산
final = index3[[0,"AADT"]].groupby(0).mean()
final

,AADT
0,
0,7493.875000
1,3573.857143
2,3430.428571
3,3430.428571
4,34270.190476
...,...
492,34270.190476
493,34542.400000
494,34270.190476


In [62]:
final2 = pd.concat([df2['번호'], final], axis = 1)
final2

,번호,AADT
0,1495,7493.875000
1,1277,3573.857143
2,1278,3430.428571
3,1279,3430.428571
4,1090,34270.190476
...,...,...
492,1089,34270.190476
493,1087,34542.400000
494,1088,34270.190476
495,1091,26748.437500


# 데이터 베이스에 올리기

In [ ]:
final2.to_sql('MainData_도은', con=db_connection, if_exists='replace', index=False)